# **Hilal Durmaz**
**Approach4:** LSA (Latent Semantic Analysis) method

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn import decomposition as dec
from sklearn import manifold as man

In [2]:
ratings = pd.read_csv("train.csv")
ratings.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [3]:
ratings.shape

(990209, 3)

In [4]:
U_matrix = ratings.pivot("userID", "movieID", "rating")
U_matrix.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#normalization
#I subtracted each value from its mean to normalize the matrix.
U_matrix_norm = U_matrix - U_matrix.mean(axis=1).values.reshape(-1,1)
U_matrix_norm.fillna(0, inplace=True)
U_matrix_norm.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.571429,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,-0.571429,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,-0.571429,0.000000,0.0,0.000000,0.428571,-0.571429,0.0,0.428571,0.0,0.428571,0.0,0.428571,0.0,0.0,-0.571429,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.288557,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-0.711443,0.0,-0.711443,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,-0.299312,0.0,0.0,0.0,0.0,-0.299312,0.0,0.0,0.0,0.0,-0.299312,0.700688,0.0,0.0,0.0,-0.299312,0.0,0.0,-1.299312,0.700688,-0.299312,0.0,0.700688,-0.299312,0.700688,0.0,-0.299312,0.0,1.700688,0.0,0.000000,0.0,0.0,-0.299312,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#train test split
m = 500
train = U_matrix_norm[:m]
test = U_matrix_norm[m:]

In [7]:
#In order to move the train data to the center, I subtracted the values in the train data from the train average.
train_mean = train.mean(axis=0)
centered_train = train - train_mean
centered_train.shape

(500, 3705)

In [8]:
#I transposed the centered train data and applied svd
U, S, VT = np.linalg.svd(centered_train.T, full_matrices=False)

In [9]:
print("U shape:", U.shape)
print("S shape:", S.shape)
print("VT shape:", VT.shape)
print("train shape:", centered_train.shape)

U shape: (3705, 500)
S shape: (500,)
VT shape: (500, 500)
train shape: (500, 3705)


In [ ]:
np.set_printoptions(suppress=True)
#I tried to find the component value by squaring and summing the values in sigma.
squares = np.array(list(map(lambda x:x**2, S)))
total = squares.sum()

for i in range(1, len(squares)):
  r = squares[:i].sum() / total
  print(i, r)


In [11]:
#236 component declares 0.90.
k = 236

U = U[:, :k]
S = S[:k]
VT = VT[:k, :]

print("U shape:", U.shape)
print("S shape:", S.shape)
print("VT shape:", VT.shape)
print("train shape:", centered_train.shape)

U shape: (3705, 236)
S shape: (236,)
VT shape: (236, 500)
train shape: (500, 3705)


In [12]:
svd = dec.TruncatedSVD(n_components=236)
temp = svd.fit_transform(centered_train)
temp.shape

(500, 236)

In [13]:
U_norm = U / np.linalg.norm(U, axis=1).reshape(-1,1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [14]:
U_norm

array([[ 0.10678396,  0.0046128 , -0.00380119, ...,  0.0462689 ,
        -0.0068094 , -0.06736973],
       [-0.0384543 , -0.05569495, -0.06355138, ...,  0.00928543,
        -0.02151196,  0.01216102],
       [-0.05122112,  0.00872742, -0.05951023, ..., -0.0105522 ,
        -0.0083863 , -0.00163742],
       ...,
       [ 0.00796039,  0.02089689, -0.00051006, ...,  0.01934112,
        -0.1085201 ,  0.05657562],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [-0.00397628, -0.00241113, -0.00187409, ..., -0.04408412,
         0.0505652 , -0.03529741]])

In [15]:
sims = U_norm.dot(U_norm.T)
sims.shape

(3705, 3705)

In [16]:
a = np.isnan(sims)
sims[a] = -1
sims[:3]

array([[ 1.        , -0.02234756,  0.0655853 , ..., -0.08128713,
        -1.        , -0.00809343],
       [-0.02234756,  1.        ,  0.04865898, ..., -0.03452322,
        -1.        ,  0.04919964],
       [ 0.0655853 ,  0.04865898,  1.        , ...,  0.04801145,
        -1.        , -0.00037848]])

In [21]:
ratings

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4
...,...,...,...
990204,4455,1077,5
990205,3912,2700,5
990206,5187,3102,3
990207,2008,3479,5


In [23]:
ratings.movieID.value_counts()[:10] #en çok oylanan filmler

2858    3393
1196    2966
260     2963
1210    2854
480     2643
589     2629
2028    2626
2571    2564
1270    2562
593     2546
Name: movieID, dtype: int64

In [25]:
movieID = 2858
simIndices = sims[movieID].argsort()[::-1][:10]
simIndices

array([2858, 1998, 1519, 2376, 2103, 1557, 3562, 3659, 1324,  898])

In [27]:
test.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
501,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,0.0,0.000000,0.0,-0.752119,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.247881,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.247881,0.0,0.247881,0.0,0.0,0.247881,0.0,...,0.0,-0.752119,0.247881,-0.752119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.752119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
505,0.0,-0.920128,0.0,0.000000,0.0,2.079872,0.0,0.0,0.0,-0.920128,1.079872,0.0,0.0,-0.920128,0.0,0.079872,0.000000,0.0,0.0,0.0,0.079872,-0.920128,-1.920128,0.079872,1.079872,0.0,0.0,0.0,1.079872,0.0,-0.920128,1.079872,0.0,0.079872,0.0,0.079872,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
myPreds = []
realLabels = []

for userId in test.index:
  print(userId)
  if userId == 515:
    break
  for movieId in test.columns:
    if not np.isnan(test.loc[userId, movieId]):
      new_user = test.loc[userId:userId, :]
      new_user_centered = new_user - train_mean

      new_user_red = svd.transform(new_user_centered)
      idx=list(test.columns).index(movieId)
      #I calculated how much the user would find a different prediction from the average.
      fark=new_user_red.dot(svd.components_.T[idx]) #multiplication of U*S vector with the transpose of V
      user = test.dropna(subset=[userId],axis=1)
      avg=user.loc[userId:userId, :].mean(axis=1).squeeze()
      rate=int(fark)+int(avg)
      myPreds.append(rate)

      real_label = test.loc[userId, movieId]
      realLabels.append(real_label)



501
502
503
504
505
506
507
508
509
510
511
512
513
514
515


In [49]:
from sklearn import metrics as met
met.mean_squared_error(realLabels, myPreds, squared=False)

0.2204030231699764

# Prediction

In [50]:
test = pd.read_csv("test_without_labels.csv")
test.head()

,IDs
0,629_2683
1,601_904
2,5673_3717
3,5616_1721
4,1099_3697


In [51]:
x=list(test.IDs.values)
userid=[]
movieId=[]

x=list(test.IDs.values)
for i in range(test.shape[0]):
  user,movie=x[i].split("_")
  userid.append(user)
  movieId.append(movie)


d = {'UserId':userid , 'MovieId': movieId}
df_Test = pd.DataFrame(data=d)

In [52]:
df_Test.head()

,UserId,MovieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697


In [54]:
U_matrix_norm_mean = U_matrix_norm.mean(axis=0)
centered_U_matrix_ = U_matrix_norm - U_matrix_norm_mean
centered_U_matrix_.shape

(6040, 3705)

In [55]:
U, S, VT = np.linalg.svd(centered_U_matrix_.T, full_matrices=False)
print("U shape:", U.shape)
print("S shape:", S.shape)
print("VT shape:", VT.shape)
print("train shape:", centered_U_matrix_.shape)

U shape: (3705, 3705)
S shape: (3705,)
VT shape: (3705, 6040)
train shape: (6040, 3705)


In [ ]:
np.set_printoptions(suppress=True)

squares = np.array(list(map(lambda x:x**2, S)))
total = squares.sum()

for i in range(1, len(squares)):
  r = squares[:i].sum() / total
  print(i, r)

  #1000 

In [58]:
k = 1000

U = U[:, :k]
S = S[:k]
VT = VT[:k, :]

print("U shape:", U.shape)
print("S shape:", S.shape)
print("VT shape:", VT.shape)
print("train shape:", centered_U_matrix_.shape)

U shape: (3705, 1000)
S shape: (1000,)
VT shape: (1000, 6040)
train shape: (6040, 3705)


In [60]:
svd = dec.TruncatedSVD(n_components=1000)
temp = svd.fit_transform(centered_U_matrix_)
temp.shape

(6040, 1000)

In [68]:
df_Test

,UserId,MovieId
0,629,2683
1,601,904
2,5673,3717
3,5616,1721
4,1099,3697
...,...,...
9994,4198,3671
9995,566,3148
9996,5531,2105
9997,4650,377


In [ ]:

myPreds = []
for i in range(len(df_Test)):
  userId=int(df_Test.UserId[i])
  movieId=int(df_Test.MovieId[i])
  new_user = U_matrix_norm.loc[userId:userId, :]
  new_user_centered = new_user - U_matrix_norm_mean
  new_user_red = svd.transform(new_user_centered)
  idx=list(U_matrix_norm.columns).index(movieId)
  fark=new_user_red.dot(svd.components_.T[idx])
  user = U_matrix.dropna(subset=[userId],axis=1)
  avg=user.loc[userId:userId, :].mean(axis=1).squeeze()
  rate=fark+avg
  rate=float(rate)
  myPreds.append(rate)
  print("rate - ",rate,"mypred- ",len(myPreds))

      

In [106]:
submission = pd.DataFrame({"IDs":list(test.IDs.values), "rating":myPreds})
submission.to_csv("submission2_apc4.csv", index = None)


In [107]:
from google.colab import files

files.download('submission2_apc4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>